# Computing Centralizers of third order operators

In [1]:
import sys
sys.path.insert(0,"..") # salgebra is here

from dalgebra import *
%display latex

In this notebook we show how to use `dalgebra` for computing centralizers (or at least, the conditions to obtain a centralizer) for some operators. This is based on results from 
* [6] G. Wilson, _Algebraic curves and soluton equations_, Geometry Today, 1985, pp. 303-329,

and also in some current and recent reasearch from:
* [3] J.J. Morales-Ruiz, S.L. Rueda and M.A. Zurro, _Factorization of KdV Schrödinger operators using differential subresultants_, Adv. Appl. Math. **120** (2020), 102065
* [4] E. Oreviato, Sl.L. Rueda and M.A. Zurro, _Commuting Ordinary Differential OPerators and the Dixmier Test_, "SIGMA (Symmetry, integrability and Geometry: Methods and Application)" **15** (2019) no. 101, 23 pp.

## The problem

Let us consider a linear differential operator of the form:
$$L = \partial^n  + u_{n-1}\partial^{n-2} + \ldots + u_1\partial + u_0,$$
where the variables $u_*$ are in some differential field $K$ with constant field within $\mathbb{C}$. The centralizers of these operators have quotien fields that can be seen as affine rings over curves. This can be used to described and understand better the centralizers of $L$ and some of its properties.

**Definition** _(almost commuting)_: let $A,B \in K[\partial]$ be of order $n$ and $m$ respectively. We say that $B$ almost commute with $A$ if $\text{ord}([A,B]) \leq n-2$.

By definition, two operators commute when $[A,B] = 0$ and, in general, this commutator have order $n+m-1$. Hence, if the order of the commutator is _so small_ that is smaller than $n-1$, then we consider the operator $B$ _almost commute_ with $A$. 

**_Remark_**: _almost commuting_ is not a symmetric property: if $\text{ord}([A,B]) = n-2$ and $n > m$ then $B$ almost commutes with $A$ but $A$ does not almost commute with $B$.

### Almost commuting operators: their structure

**Lemma** let $A \in K[\partial]$. Then the set of $B$ such that $B$ almost commute with $A$ is a $\mathbb{C}$-vector space.

_Proof:_ this is trivial using the properties of $[\cdot,\cdot]$:
* $[A, B+C] = AB + AC - BA - CA = [A,B] + [A,C]$. Hence $\text{ord}([A,B+C]) \leq \max\{\text{ord}([A,B]), \text{ord}([A,C])\} \leq n-2$.
* If $\partial(c) = 0$, then $[A,cB] = c[A,B]$. Hence, $\text{ord}([A,cB]) = \text{ord}([A,B]) \leq n-2$.

As it usually happens with this type of vector spaces of operators, we can consider them sliced into the orders of the operators. In particular, we have that the sets
$$AC_m(A) = \left\{B \in K[\partial]\ :\ \text{$B$ almost commute with $A$ and $\text{ord}(B) \leq m$}\right\}$$
are subspaces of the space of almost commuting operators.

It was shown in [6] that the vector spaces $AC_m(A)$ has exactly dimension $m$. This, together with the fact that $AC_m(A) \subset AC_{m+1}(A)$, means that there is an operator of order exactly $m$ that almost commute with $A$. This operator can be uniquely defined if provided more properties.

**_Remark_**: if $B$ commutes with $A$, then $B$ almost commute with $A$.

### Computing a basis of $AC_m(L)$

Let's go back to the operator $L$ that was of our interest. We can create the operator in a ring $\mathbb{Q}\{u_0,\ldots,u_{n-2},Y_0,\ldots,Y_{m-2},z\}$, then create the operator $L$ and $P_m$ of the given orders and then solve the corresponding system to have $P_m$ almost commuting with $L$. The following method will do the trick. This method returns:

* The operator $P_m$ in the ring $\mathbb{Q}\{u_0,\ldots,u_{n-2}, z\}$.
* The commutator $[L,P_m] = T_0 + T_1\partial + T_{n-2}\partial^{n-2}$, where $T_0,T_1 \in \mathbb{Q}\{u_0,\ldots,u_{n-2}\}$.

In [2]:
def quasi_commutting_generic(n, m):
    names_u = ["u"] if n == 2 else [f"u{i}" for i in range(n-1)]
    names_Y = ["Y"] if m == 2 else [f"Y{j}" for j in range(m-1)]
    R = DifferentialPolynomialRing(QQ, names_u + names_Y + ["z"])
    Y, u, z = R.gens()[:m-1], R.gens()[m-1:m+n-2], R.gens()[-1]

    ## Creating the two operators L and P
    L = -z[n] + sum(z[i]*u[i][0] for i in range(n-1))
    P = z[m] + sum(z[i]*Y[i][0] for i in range(m-1))

    ## Computing commutator
    commutator = L(z=P) - P(z=L)
    system = DifferentialSystem([commutator.coefficient(z[i]) for i in range(n-1, commutator.order(z)+1)], variables=Y)
    sols = system.solve_linear()
    
    ## Output of method
    RO_u = DifferentialPolynomialRing(QQ, names_u)
    RO = DifferentialPolynomialRing(RO_u, ["z"])
    P_eval = RO(P(**{k.variable_name() : v for (k,v) in sols.items()}))
    commutator_eval = commutator(**{k.variable_name() : v for (k,v) in sols.items()})
    T = [RO_u(commutator_eval.coefficient(z[i])) for i in range(n-1)]
    
    return P_eval, tuple(T)

In [29]:
def get_sequence(n, bound):
    sequence = []
    for i in range(n, bound+1):
        P, T = quasi_commutting_generic(n,i)
        
        print("-------------------------------------------------------------------------------------------------------------------")
        print(f"Computed the almost commutator of order {i}")
        show(f"P_{i} = " + latex(P))

        if all(t == 0 for t in T):
            print("... This commutes with L")
            sequence.append((P, tuple([])))
        else:
            ltx_code = r"\left\{\begin{array}{rcl}"
            ltx_code += r"\\".join(r"f_{" + f"{i//n},{j}" + r"} & = & " + latex(T[j]) for j in range(len(T)))
            ltx_code += r"\end{array}\right."
            sequence.append((P, tuple([t for t in T if t != 0])))
            show(LatexExpr(ltx_code))
    return sequence

## Recovering the kdv sequence

In [32]:
%time seq=get_sequence(2, 10)

-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 2


P_2 = -u_{0} z_{0} + 1 z_{2}

... This commutes with L
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 3


P_3 = \left(-\frac{3}{4}\right) u_{1} z_{0} + \left(-\frac{3}{2}\right) u_{0} z_{1} + 1 z_{3}

\left\{\begin{array}{rcl}f_{1,0} & = & \frac{3}{2} u_{1} u_{0} + \left(-\frac{1}{4}\right) u_{3}\end{array}\right.

...

P_9 = \frac{315}{64} u_{1} u_{0}^{3} z_{0} + \frac{315}{128} u_{0}^{4} z_{1} + \left(-\frac{945}{128}\right) u_{3} u_{0}^{2} z_{0} + \left(-\frac{945}{32}\right) u_{2} u_{1} u_{0} z_{0} + \left(-\frac{1575}{64}\right) u_{2} u_{0}^{2} z_{1} + \left(-\frac{945}{128}\right) u_{1}^{3} z_{0} + \left(-\frac{2205}{64}\right) u_{1}^{2} u_{0} z_{1} + \left(-\frac{945}{32}\right) u_{1} u_{0}^{2} z_{2} + \left(-\frac{105}{16}\right) u_{0}^{3} z_{3} + \frac{567}{128} u_{5} u_{0} z_{0} + \frac{1743}{128} u_{4} u_{1} z_{0} + \frac{1449}{64} u_{4} u_{0} z_{1} + \frac{3045}{128} u_{3} u_{2} z_{0} + \frac{1827}{32} u_{3} u_{1} z_{1} + \frac{1575}{32} u_{3} u_{0} z_{2} + \frac{5061}{128} u_{2}^{2} z_{1} + \frac{1575}{16} u_{2} u_{1} z_{2} + \frac{945}{16} u_{2} u_{0} z_{3} + \frac{1365}{32} u_{1}^{2} z_{3} + \frac{315}{8} u_{1} u_{0} z_{4} + \frac{63}{8} u_{0}^{2} z_{5} + \left(-\frac{255}{256}\right) u_{7} z_{0} + \left(-\frac{897}{128}\right) u_{6} z_{1} + \left(-\frac{1407}{64}\right) u_{5} z_{2} + \left(-\frac{1281}{32}\right) u_{4} z_{3} + \left(-\frac{735}{16}\right) u_{3} z_{4} + \left(-\frac{273}{8}\right) u_{2} z_{5} + \left(-\frac{63}{4}\right) u_{1} z_{6} + \left(-\frac{9}{2}\right) u_{0} z_{7} + 1 z_{9}

\left\{\begin{array}{rcl}f_{4,0} & = & \left(-\frac{315}{128}\right) u_{1} u_{0}^{4} + \frac{105}{64} u_{3} u_{0}^{3} + \frac{315}{32} u_{2} u_{1} u_{0}^{2} + \frac{315}{64} u_{1}^{3} u_{0} + \left(-\frac{63}{128}\right) u_{5} u_{0}^{2} + \left(-\frac{189}{64}\right) u_{4} u_{1} u_{0} + \left(-\frac{315}{64}\right) u_{3} u_{2} u_{0} + \left(-\frac{483}{128}\right) u_{3} u_{1}^{2} + \left(-\frac{651}{128}\right) u_{2}^{2} u_{1} + \frac{9}{128} u_{7} u_{0} + \frac{9}{32} u_{6} u_{1} + \frac{21}{32} u_{5} u_{2} + \frac{63}{64} u_{4} u_{3} + \left(-\frac{1}{256}\right) u_{9}\end{array}\right.

-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 10


P_10 = -u_{0}^{5} z_{0} + 10 u_{2} u_{0}^{3} z_{0} + 20 u_{1}^{2} u_{0}^{2} z_{0} + 20 u_{1} u_{0}^{3} z_{1} + 5 u_{0}^{4} z_{2} + \left(-10\right) u_{4} u_{0}^{2} z_{0} + \left(-50\right) u_{3} u_{1} u_{0} z_{0} + \left(-40\right) u_{3} u_{0}^{2} z_{1} + \left(-35\right) u_{2}^{2} u_{0} z_{0} + \left(-52\right) u_{2} u_{1}^{2} z_{0} + \left(-160\right) u_{2} u_{1} u_{0} z_{1} + \left(-70\right) u_{2} u_{0}^{2} z_{2} + \left(-40\right) u_{1}^{3} z_{1} + \left(-100\right) u_{1}^{2} u_{0} z_{2} + \left(-60\right) u_{1} u_{0}^{2} z_{3} + \left(-10\right) u_{0}^{3} z_{4} + 5 u_{6} u_{0} z_{0} + 18 u_{5} u_{1} z_{0} + 30 u_{5} u_{0} z_{1} + 38 u_{4} u_{2} z_{0} + 92 u_{4} u_{1} z_{1} + 80 u_{4} u_{0} z_{2} + 24 u_{3}^{2} z_{0} + 160 u_{3} u_{2} z_{1} + 202 u_{3} u_{1} z_{2} + 120 u_{3} u_{0} z_{3} + 139 u_{2}^{2} z_{2} + 240 u_{2} u_{1} z_{3} + 110 u_{2} u_{0} z_{4} + 80 u_{1}^{2} z_{4} + 60 u_{1} u_{0} z_{5} + 10 u_{0}^{2} z_{6} - u_{8} z_{0} + \left(-8\right) u_{7} z_{1} + \left(-29\right) u_{6} z_{2} + \left(-62\right) u_{5} z_{3} + \left(-86\right) u_{4} z_{4} + \left(-80\right) u_{3} z_{5} + \left(-50\right) u_{2} z_{6} + \left(-20\right) u_{1} z_{7} + \left(-5\right) u_{0} z_{8} + 1 z_{10}

... This commutes with L
CPU times: user 6.4 s, sys: 10.1 ms, total: 6.41 s
Wall time: 6.41 s


## Getting the sequence for operator of order 3

In [33]:
%time seq=get_sequence(3, 10)

-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 3


P_3 = -\mathit{u0}_{0} z_{0} - \mathit{u1}_{0} z_{1} + 1 z_{3}

... This commutes with L
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 4


P_4 = \frac{2}{9} \mathit{u1}_{0}^{2} z_{0} + \left(-\frac{2}{3}\right) \mathit{u0}_{1} z_{0} + \left(-\frac{4}{3}\right) \mathit{u0}_{0} z_{1} + \left(-\frac{2}{9}\right) \mathit{u1}_{2} z_{0} + \left(-\frac{2}{3}\right) \mathit{u1}_{1} z_{1} + \left(-\frac{4}{3}\right) \mathit{u1}_{0} z_{2} + 1 z_{4}

\left\{\begin{array}{rcl}f_{1,0} & = & \frac{4}{9} \mathit{u1}_{1} \mathit{u1}_{0}^{2} + \frac{2}{3} \mathit{u0}_{2} \mathit{u1}_{0} + \frac{4}{3} \mathit{u0}_{1} \mathit{u0}_{0} + \frac{2}{3} \mathit{u0}_{1} \mathit{u1}_{1} + \left(-\frac{2}{3}\right) \mathit{u1}_{3} \mathit{u1}_{0} + \left(-\frac{4}{3}\right) \mathit{u1}_{2} \mathit{u1}_{1} + \left(-\frac{1}{3}\right) \mathit{u0}_{4} + \frac{2}{9} \mathit{u1}_{5}\\f_{1,1} & = & \frac{4}{3} \mathit{u0}_{1} \mathit{u1}_{0} + \frac{4}{3} \mathit{u0}_{0} \mathit{u1}_{1} + \left(-\frac{2}{3}\right) \mathit{u1}_{2} \mathit{u1}_{0} + \left(-\frac{2}{3}\right) \mathit{u1}_{1}^{2} + \left(-\frac{2}{3}\right) \mathit{u0}_{3} + \frac{1}{3} \mathit{u1}_{4}\end{array}\right.

...

P_9 = -\mathit{u0}_{2} \mathit{u1}_{0}^{2} z_{0} + \left(-3\right) \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{0} z_{0} - \mathit{u0}_{1} \mathit{u1}_{1} \mathit{u1}_{0} z_{0} + \left(-3\right) \mathit{u0}_{1} \mathit{u1}_{0}^{2} z_{1} - \mathit{u0}_{0}^{3} z_{0} + \left(-3\right) \mathit{u0}_{0}^{2} \mathit{u1}_{0} z_{1} + \left(-3\right) \mathit{u0}_{0} \mathit{u1}_{1} \mathit{u1}_{0} z_{1} + \left(-3\right) \mathit{u0}_{0} \mathit{u1}_{0}^{2} z_{2} - \mathit{u1}_{2} \mathit{u1}_{0}^{2} z_{1} - \mathit{u1}_{1}^{2} \mathit{u1}_{0} z_{1} + \left(-3\right) \mathit{u1}_{1} \mathit{u1}_{0}^{2} z_{2} - \mathit{u1}_{0}^{3} z_{3} + 2 \mathit{u0}_{4} \mathit{u1}_{0} z_{0} + 3 \mathit{u0}_{3} \mathit{u0}_{0} z_{0} + 3 \mathit{u0}_{3} \mathit{u1}_{1} z_{0} + 9 \mathit{u0}_{3} \mathit{u1}_{0} z_{1} + 6 \mathit{u0}_{2} \mathit{u0}_{1} z_{0} + 9 \mathit{u0}_{2} \mathit{u0}_{0} z_{1} + 3 \mathit{u0}_{2} \mathit{u1}_{2} z_{0} + 12 \mathit{u0}_{2} \mathit{u1}_{1} z_{1} + 15 \mathit{u0}_{2} \mathit{u1}_{0} z_{2} + 6 \mathit{u0}_{1}^{2} z_{1} + 9 \mathit{u0}_{1} \mathit{u0}_{0} z_{2} + 1 \mathit{u0}_{1} \mathit{u1}_{3} z_{0} + 9 \mathit{u0}_{1} \mathit{u1}_{2} z_{1} + 15 \mathit{u0}_{1} \mathit{u1}_{1} z_{2} + 12 \mathit{u0}_{1} \mathit{u1}_{0} z_{3} + 3 \mathit{u0}_{0}^{2} z_{3} + 3 \mathit{u0}_{0} \mathit{u1}_{3} z_{1} + 9 \mathit{u0}_{0} \mathit{u1}_{2} z_{2} + 9 \mathit{u0}_{0} \mathit{u1}_{1} z_{3} + 6 \mathit{u0}_{0} \mathit{u1}_{0} z_{4} + 2 \mathit{u1}_{4} \mathit{u1}_{0} z_{1} + 4 \mathit{u1}_{3} \mathit{u1}_{1} z_{1} + 9 \mathit{u1}_{3} \mathit{u1}_{0} z_{2} + 3 \mathit{u1}_{2}^{2} z_{1} + 15 \mathit{u1}_{2} \mathit{u1}_{1} z_{2} + 15 \mathit{u1}_{2} \mathit{u1}_{0} z_{3} + 9 \mathit{u1}_{1}^{2} z_{3} + 12 \mathit{u1}_{1} \mathit{u1}_{0} z_{4} + 3 \mathit{u1}_{0}^{2} z_{5} - \mathit{u0}_{6} z_{0} + \left(-6\right) \mathit{u0}_{5} z_{1} + \left(-15\right) \mathit{u0}_{4} z_{2} + \left(-21\right) \mathit{u0}_{3} z_{3} + \left(-18\right) \mathit{u0}_{2} z_{4} + \left(-9\right) \mathit{u0}_{1} z_{5} + \left(-3\right) \mathit{u0}_{0} z_{6} - \mathit{u1}_{6} z_{1} + \left(-6\right) \mathit{u1}_{5} z_{2} + \left(-15\right) \mathit{u1}_{4} z_{3} + \left(-21\right) \mathit{u1}_{3} z_{4} + \left(-18\right) \mathit{u1}_{2} z_{5} + \left(-9\right) \mathit{u1}_{1} z_{6} + \left(-3\right) \mathit{u1}_{0} z_{7} + 1 z_{9}

... This commutes with L
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 10


P_10 = \frac{14}{729} \mathit{u1}_{0}^{5} z_{0} + \frac{70}{243} \mathit{u0}_{1} \mathit{u1}_{0}^{3} z_{0} + \frac{70}{81} \mathit{u0}_{0}^{2} \mathit{u1}_{0}^{2} z_{0} + \frac{140}{243} \mathit{u0}_{0} \mathit{u1}_{0}^{3} z_{1} + \left(-\frac{140}{729}\right) \mathit{u1}_{2} \mathit{u1}_{0}^{3} z_{0} + \left(-\frac{70}{243}\right) \mathit{u1}_{1}^{2} \mathit{u1}_{0}^{2} z_{0} + \frac{70}{243} \mathit{u1}_{1} \mathit{u1}_{0}^{3} z_{1} + \frac{35}{243} \mathit{u1}_{0}^{4} z_{2} + \left(-\frac{140}{81}\right) \mathit{u0}_{3} \mathit{u1}_{0}^{2} z_{0} + \left(-\frac{140}{27}\right) \mathit{u0}_{2} \mathit{u0}_{0} \mathit{u1}_{0} z_{0} + \left(-\frac{280}{81}\right) \mathit{u0}_{2} \mathit{u1}_{1} \mathit{u1}_{0} z_{0} + \left(-\frac{490}{81}\right) \mathit{u0}_{2} \mathit{u1}_{0}^{2} z_{1} + \left(-\frac{280}{81}\right) \mathit{u0}_{1}^{2} \mathit{u1}_{0} z_{0} + \left(-\frac{70}{27}\right) \mathit{u0}_{1} \mathit{u0}_{0}^{2} z_{0} + \left(-\frac{280}{81}\right) \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{1} z_{0} + \left(-\frac{280}{27}\right) \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{0} z_{1} + \left(-\frac{140}{81}\right) \mathit{u0}_{1} \mathit{u1}_{2} \mathit{u1}_{0} z_{0} + \left(-\frac{70}{81}\right) \mathit{u0}_{1} \mathit{u1}_{1}^{2} z_{0} + \left(-\frac{280}{27}\right) \mathit{u0}_{1} \mathit{u1}_{1} \mathit{u1}_{0} z_{1} + \left(-\frac{70}{9}\right) \mathit{u0}_{1} \mathit{u1}_{0}^{2} z_{2} + \left(-\frac{140}{81}\right) \mathit{u0}_{0}^{3} z_{1} + \left(-\frac{70}{81}\right) \mathit{u0}_{0}^{2} \mathit{u1}_{2} z_{0} + \left(-\frac{70}{27}\right) \mathit{u0}_{0}^{2} \mathit{u1}_{1} z_{1} + \left(-\frac{140}{27}\right) \mathit{u0}_{0}^{2} \mathit{u1}_{0} z_{2} + \left(-\frac{140}{27}\right) \mathit{u0}_{0} \mathit{u1}_{2} \mathit{u1}_{0} z_{1} + \left(-\frac{70}{27}\right) \mathit{u0}_{0} \mathit{u1}_{1}^{2} z_{1} + \left(-\frac{280}{27}\right) \mathit{u0}_{0} \mathit{u1}_{1} \mathit{u1}_{0} z_{2} + \left(-\frac{140}{27}\right) \mathit{u0}_{0} \mathit{u1}_{0}^{2} z_{3} + \frac{14}{81} \mathit{u1}_{4} \mathit{u1}_{0}^{2} z_{0} + \frac{154}{243} \mathit{u1}_{3} \mathit{u1}_{1} \mathit{u1}_{0} z_{0} + \left(-\frac{140}{81}\right) \mathit{u1}_{3} \mathit{u1}_{0}^{2} z_{1} + \frac{98}{243} \mathit{u1}_{2}^{2} \mathit{u1}_{0} z_{0} + \frac{14}{27} \mathit{u1}_{2} \mathit{u1}_{1}^{2} z_{0} + \left(-\frac{140}{27}\right) \mathit{u1}_{2} \mathit{u1}_{1} \mathit{u1}_{0} z_{1} + \left(-\frac{490}{81}\right) \mathit{u1}_{2} \mathit{u1}_{0}^{2} z_{2} + \left(-\frac{70}{81}\right) \mathit{u1}_{1}^{3} z_{1} + \left(-\frac{560}{81}\right) \mathit{u1}_{1}^{2} \mathit{u1}_{0} z_{2} + \left(-\frac{70}{9}\right) \mathit{u1}_{1} \mathit{u1}_{0}^{2} z_{3} + \left(-\frac{140}{81}\right) \mathit{u1}_{0}^{3} z_{4} + \frac{196}{81} \mathit{u0}_{5} \mathit{u1}_{0} z_{0} + \frac{280}{81} \mathit{u0}_{4} \mathit{u0}_{0} z_{0} + \frac{392}{81} \mathit{u0}_{4} \mathit{u1}_{1} z_{0} + \frac{1022}{81} \mathit{u0}_{4} \mathit{u1}_{0} z_{1} + \frac{238}{27} \mathit{u0}_{3} \mathit{u0}_{1} z_{0} + \frac{350}{27} \mathit{u0}_{3} \mathit{u0}_{0} z_{1} + \frac{518}{81} \mathit{u0}_{3} \mathit{u1}_{2} z_{0} + \frac{203}{9} \mathit{u0}_{3} \mathit{u1}_{1} z_{1} + \frac{245}{9} \mathit{u0}_{3} \mathit{u1}_{0} z_{2} + \frac{518}{81} \mathit{u0}_{2}^{2} z_{0} + \frac{700}{27} \mathit{u0}_{2} \mathit{u0}_{1} z_{1} + \frac{560}{27} \mathit{u0}_{2} \mathit{u0}_{0} z_{2} + \frac{322}{81} \mathit{u0}_{2} \mathit{u1}_{3} z_{0} + \frac{2023}{81} \mathit{u0}_{2} \mathit{u1}_{2} z_{1} + \frac{1085}{27} \mathit{u0}_{2} \mathit{u1}_{1} z_{2} + \frac{280}{9} \mathit{u0}_{2} \mathit{u1}_{0} z_{3} + \frac{385}{27} \mathit{u0}_{1}^{2} z_{2} + \frac{140}{9} \mathit{u0}_{1} \mathit{u0}_{0} z_{3} + \frac{28}{27} \mathit{u0}_{1} \mathit{u1}_{4} z_{0} + \frac{1022}{81} \mathit{u0}_{1} \mathit{u1}_{3} z_{1} + \frac{875}{27} \mathit{u0}_{1} \mathit{u1}_{2} z_{2} + \frac{910}{27} \mathit{u0}_{1} \mathit{u1}_{1} z_{3} + \frac{175}{9} \mathit{u0}_{1} \mathit{u1}_{0} z_{4} + \frac{35}

\left\{\begin{array}{rcl}f_{3,0} & = & \frac{70}{729} \mathit{u1}_{1} \mathit{u1}_{0}^{5} + \frac{35}{243} \mathit{u0}_{2} \mathit{u1}_{0}^{4} + \frac{280}{243} \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{0}^{3} + \frac{140}{243} \mathit{u0}_{1} \mathit{u1}_{1} \mathit{u1}_{0}^{3} + \frac{140}{81} \mathit{u0}_{0}^{2} \mathit{u1}_{1} \mathit{u1}_{0}^{2} + \left(-\frac{70}{243}\right) \mathit{u1}_{3} \mathit{u1}_{0}^{4} + \left(-\frac{560}{243}\right) \mathit{u1}_{2} \mathit{u1}_{1} \mathit{u1}_{0}^{3} + \left(-\frac{140}{81}\right) \mathit{u1}_{1}^{3} \mathit{u1}_{0}^{2} + \left(-\frac{70}{243}\right) \mathit{u0}_{4} \mathit{u1}_{0}^{3} + \left(-\frac{140}{81}\right) \mathit{u0}_{3} \mathit{u0}_{0} \mathit{u1}_{0}^{2} + \left(-\frac{140}{81}\right) \mathit{u0}_{3} \mathit{u1}_{1} \mathit{u1}_{0}^{2} + \left(-\frac{280}{81}\right) \mathit{u0}_{2} \mathit{u0}_{1} \mathit{u1}_{0}^{2} + \frac{70}{27} \mathit{u0}_{2} \mathit{u0}_{0}^{2} \mathit{u1}_{0} + \left(-\frac{700}{81}\right) \mathit{u0}_{2} \mathit{u0}_{0} \mathit{u1}_{1} \mathit{u1}_{0} + \left(-\frac{140}{81}\right) \mathit{u0}_{2} \mathit{u1}_{2} \mathit{u1}_{0}^{2} + \left(-\frac{70}{27}\right) \mathit{u0}_{2} \mathit{u1}_{1}^{2} \mathit{u1}_{0} + \frac{140}{27} \mathit{u0}_{1}^{2} \mathit{u0}_{0} \mathit{u1}_{0} + \left(-\frac{560}{81}\right) \mathit{u0}_{1}^{2} \mathit{u1}_{1} \mathit{u1}_{0} + \frac{140}{81} \mathit{u0}_{1} \mathit{u0}_{0}^{3} + \frac{70}{27} \mathit{u0}_{1} \mathit{u0}_{0}^{2} \mathit{u1}_{1} + \left(-\frac{280}{27}\right) \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{2} \mathit{u1}_{0} + \left(-\frac{70}{9}\right) \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{1}^{2} + \left(-\frac{70}{81}\right) \mathit{u0}_{1} \mathit{u1}_{3} \mathit{u1}_{0}^{2} + \left(-\frac{280}{81}\right) \mathit{u0}_{1} \mathit{u1}_{2} \mathit{u1}_{1} \mathit{u1}_{0} + \left(-\frac{70}{81}\right) \mathit{u0}_{1} \mathit{u1}_{1}^{3} + \left(-\frac{70}{27}\right) \mathit{u0}_{0}^{2} \mathit{u1}_{3} \mathit{u1}_{0} + \left(-\frac{140}{27}\right) \mathit{u0}_{0}^{2} \mathit{u1}_{2} \mathit{u1}_{1} + \frac{266}{729} \mathit{u1}_{5} \mathit{u1}_{0}^{3} + \frac{266}{81} \mathit{u1}_{4} \mathit{u1}_{1} \mathit{u1}_{0}^{2} + \frac{1330}{243} \mathit{u1}_{3} \mathit{u1}_{2} \mathit{u1}_{0}^{2} + \frac{700}{81} \mathit{u1}_{3} \mathit{u1}_{1}^{2} \mathit{u1}_{0} + \frac{2870}{243} \mathit{u1}_{2}^{2} \mathit{u1}_{1} \mathit{u1}_{0} + \frac{1540}{243} \mathit{u1}_{2} \mathit{u1}_{1}^{3} + \frac{7}{27} \mathit{u0}_{6} \mathit{u1}_{0}^{2} + \frac{70}{81} \mathit{u0}_{5} \mathit{u0}_{0} \mathit{u1}_{0} + \frac{133}{81} \mathit{u0}_{5} \mathit{u1}_{1} \mathit{u1}_{0} + \frac{217}{81} \mathit{u0}_{4} \mathit{u0}_{1} \mathit{u1}_{0} + \left(-\frac{35}{27}\right) \mathit{u0}_{4} \mathit{u0}_{0}^{2} + \frac{280}{81} \mathit{u0}_{4} \mathit{u0}_{0} \mathit{u1}_{1} + \frac{70}{27} \mathit{u0}_{4} \mathit{u1}_{2} \mathit{u1}_{0} + \frac{175}{81} \mathit{u0}_{4} \mathit{u1}_{1}^{2} + \frac{385}{81} \mathit{u0}_{3} \mathit{u0}_{2} \mathit{u1}_{0} + \left(-\frac{70}{9}\right) \mathit{u0}_{3} \mathit{u0}_{1} \mathit{u0}_{0} + \frac{763}{81} \mathit{u0}_{3} \mathit{u0}_{1} \mathit{u1}_{1} + \frac{560}{81} \mathit{u0}_{3} \mathit{u0}_{0} \mathit{u1}_{2} + \frac{175}{81} \mathit{u0}_{3} \mathit{u1}_{3} \mathit{u1}_{0} + \frac{140}{27} \mathit{u0}_{3} \mathit{u1}_{2} \mathit{u1}_{1} + \left(-\frac{140}{27}\right) \mathit{u0}_{2}^{2} \mathit{u0}_{0} + \frac{175}{27} \mathit{u0}_{2}^{2} \mathit{u1}_{1} + \left(-\frac{245}{27}\right) \mathit{u0}_{2} \mathit{u0}_{1}^{2} + \frac{1232}{81} \mathit{u0}_{2} \mathit{u0}_{1} \mathit{u1}_{2} + \frac{70}{9} \mathit{u0}_{2} \mathit{u0}_{0} \mathit{u1}_{3} + \frac{28}{27} \mathit{u0}_{2} \mathit{u1}_{4} \mathit{u1}_{0} + \frac{245}{81} \mathit{u0}_{2} \mathit{u1}_{3} \mathit{u1}_{1} + \frac{175}{81} \mathit{u0}_{2} \mathit{u1}_{2}^{2} + \frac{518}{81} \mathit{u0}_{1}^{2} \mathit{u1}_{3} + \frac{140}{27} \mathit{u0}_{1} \mathit{u0}_{0} \mathit{u1}_{4} + \frac{28}{81} \math

CPU times: user 15.7 s, sys: 70.2 ms, total: 15.7 s
Wall time: 15.7 s


## Getting the sequence for operator of order 5

In [34]:
%time seq=get_sequence(5, 10)

-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 5


P_5 = -\mathit{u0}_{0} z_{0} - \mathit{u1}_{0} z_{1} - \mathit{u2}_{0} z_{2} - \mathit{u3}_{0} z_{3} + 1 z_{5}

... This commutes with L
-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 6


P_6 = \frac{4}{125} \mathit{u3}_{0}^{3} z_{0} + \frac{6}{25} \mathit{u1}_{0} \mathit{u3}_{0} z_{0} + \left(-\frac{3}{25}\right) \mathit{u2}_{1} \mathit{u3}_{0} z_{0} + \frac{3}{25} \mathit{u2}_{0}^{2} z_{0} + \left(-\frac{6}{25}\right) \mathit{u2}_{0} \mathit{u3}_{1} z_{0} + \frac{6}{25} \mathit{u2}_{0} \mathit{u3}_{0} z_{1} + \left(-\frac{3}{25}\right) \mathit{u3}_{2} \mathit{u3}_{0} z_{0} + \left(-\frac{1}{25}\right) \mathit{u3}_{1}^{2} z_{0} + \left(-\frac{3}{25}\right) \mathit{u3}_{1} \mathit{u3}_{0} z_{1} + \frac{3}{25} \mathit{u3}_{0}^{2} z_{2} + \left(-\frac{3}{5}\right) \mathit{u0}_{1} z_{0} + \left(-\frac{6}{5}\right) \mathit{u0}_{0} z_{1} + \left(-\frac{2}{5}\right) \mathit{u1}_{2} z_{0} + \left(-\frac{3}{5}\right) \mathit{u1}_{1} z_{1} + \left(-\frac{6}{5}\right) \mathit{u1}_{0} z_{2} + \frac{1}{5} \mathit{u2}_{3} z_{0} + \left(-\frac{2}{5}\right) \mathit{u2}_{2} z_{1} + \left(-\frac{3}{5}\right) \mathit{u2}_{1} z_{2} + \left(-\frac{6}{5}\right) \mathit{u2}_{0} z_{3} + \frac{1}{25} \mathit{u3}_{4} z_{0} + \frac{1}{5} \mathit{u3}_{3} z_{1} + \left(-\frac{2}{5}\right) \mathit{u3}_{2} z_{2} + \left(-\frac{3}{5}\right) \mathit{u3}_{1} z_{3} + \left(-\frac{6}{5}\right) \mathit{u3}_{0} z_{4} + 1 z_{6}

\left\{\begin{array}{rcl}f_{1,0} & = & \frac{12}{125} \mathit{u1}_{0} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \frac{12}{125} \mathit{u2}_{0} \mathit{u3}_{2} \mathit{u3}_{0}^{2} + \frac{24}{125} \mathit{u2}_{0} \mathit{u3}_{1}^{2} \mathit{u3}_{0} + \frac{12}{125} \mathit{u3}_{3} \mathit{u3}_{0}^{3} + \frac{72}{125} \mathit{u3}_{2} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \frac{24}{125} \mathit{u3}_{1}^{3} \mathit{u3}_{0} + \left(-\frac{3}{25}\right) \mathit{u0}_{2} \mathit{u3}_{0}^{2} + \left(-\frac{6}{25}\right) \mathit{u0}_{1} \mathit{u2}_{0} \mathit{u3}_{0} + \frac{3}{25} \mathit{u0}_{1} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{6}{25} \mathit{u1}_{3} \mathit{u3}_{0}^{2} + \frac{6}{25} \mathit{u1}_{2} \mathit{u2}_{0} \mathit{u3}_{0} + \frac{18}{25} \mathit{u1}_{2} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{6}{25} \mathit{u1}_{1} \mathit{u1}_{0} \mathit{u3}_{0} + \frac{12}{25} \mathit{u1}_{1} \mathit{u2}_{0} \mathit{u3}_{1} + \frac{18}{25} \mathit{u1}_{1} \mathit{u3}_{2} \mathit{u3}_{0} + \frac{6}{25} \mathit{u1}_{0}^{2} \mathit{u3}_{1} + \left(-\frac{3}{25}\right) \mathit{u1}_{0} \mathit{u2}_{2} \mathit{u3}_{0} + \frac{6}{25} \mathit{u1}_{0} \mathit{u2}_{1} \mathit{u2}_{0} + \left(-\frac{9}{25}\right) \mathit{u1}_{0} \mathit{u2}_{1} \mathit{u3}_{1} + \frac{3}{25} \mathit{u1}_{0} \mathit{u3}_{3} \mathit{u3}_{0} + \left(-\frac{1}{5}\right) \mathit{u1}_{0} \mathit{u3}_{2} \mathit{u3}_{1} + \left(-\frac{3}{25}\right) \mathit{u2}_{4} \mathit{u3}_{0}^{2} + \frac{3}{25} \mathit{u2}_{3} \mathit{u2}_{0} \mathit{u3}_{0} + \left(-\frac{3}{5}\right) \mathit{u2}_{3} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{18}{25} \mathit{u2}_{2} \mathit{u2}_{1} \mathit{u3}_{0} + \frac{6}{25} \mathit{u2}_{2} \mathit{u2}_{0}^{2} + \left(-\frac{12}{25}\right) \mathit{u2}_{2} \mathit{u2}_{0} \mathit{u3}_{1} + \left(-\frac{27}{25}\right) \mathit{u2}_{2} \mathit{u3}_{2} \mathit{u3}_{0} + \frac{6}{25} \mathit{u2}_{1}^{2} \mathit{u2}_{0} + \left(-\frac{3}{5}\right) \mathit{u2}_{1} \mathit{u2}_{0} \mathit{u3}_{2} + \left(-\frac{21}{25}\right) \mathit{u2}_{1} \mathit{u3}_{3} \mathit{u3}_{0} + \left(-\frac{6}{25}\right) \mathit{u2}_{0}^{2} \mathit{u3}_{3} + \left(-\frac{9}{25}\right) \mathit{u2}_{0} \mathit{u3}_{4} \mathit{u3}_{0} + \left(-\frac{8}{25}\right) \mathit{u2}_{0} \mathit{u3}_{3} \mathit{u3}_{1} + \left(-\frac{1}{5}\right) \mathit{u2}_{0} \mathit{u3}_{2}^{2} + \left(-\frac{27}{125}\right) \mathit{u3}_{5} \mathit{u3}_{0}^{2} + \left(-\frac{7}{5}\right) \mathit{u3}_{4} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{66}{25}\right) \mathit{u3}_{3} \mathit{u3}_{2} \mathit{u3}_{0} + \left(-\frac{48}{25}\right) \mathit{u3}_{3} \mathit{u3}_{1}^{2} + \left(-\frac{72}{25}\right) \mathit{u3}_{2}^{2} \mathit{u3}_{1} + \frac{3}{5} \mathit{u0}_{4} \mathit{u3}_{0} + \frac{3}{5} \mathit{u0}_{3} \mathit{u2}_{0} + \frac{3}{5} \mathit{u0}_{3} \mathit{u3}_{1} + \frac{3}{5} \mathit{u0}_{2} \mathit{u1}_{0} + \frac{3}{5} \mathit{u0}_{2} \mathit{u2}_{1} + \frac{2}{5} \mathit{u0}_{2} \mathit{u3}_{2} + \frac{6}{5} \mathit{u0}_{1} \mathit{u0}_{0} + \frac{3}{5} \mathit{u0}_{1} \mathit{u1}_{1} + \frac{2}{5} \mathit{u0}_{1} \mathit{u2}_{2} + \left(-\frac{1}{5}\right) \mathit{u0}_{1} \mathit{u3}_{3} + \left(-\frac{16}{25}\right) \mathit{u1}_{5} \mathit{u3}_{0} + \left(-\frac{2}{5}\right) \mathit{u1}_{4} \mathit{u2}_{0} + \left(-\frac{6}{5}\right) \mathit{u1}_{4} \mathit{u3}_{1} + \left(-\frac{2}{5}\right) \mathit{u1}_{3} \mathit{u1}_{0} + \left(-\frac{12}{5}\right) \mathit{u1}_{3} \mathit{u3}_{2} + \left(-\frac{12}{5}\right) \mathit{u1}_{2} \mathit{u3}_{3} + \left(-\frac{6}{5}\right) \mathit{u1}_{1} \mathit{u3}_{4} + \frac{1}{5} \mathit{u1}_{0} \mathit{u2}_{4} + \left(-\frac{1}{5}\right) \mathit{u1}_{0} \mathit{u3}_{5} + \frac{8}{25} \mathit{u2}_{6} \mathit{u3}_{0} + \left(-\frac{1}{25}\right) \mathit{u2}_{5} \mathit{u2}_{0} + \frac{21}{25} \mathit{u2}_{5} \mathit{u3}_{1} + \left(-\frac{6}{5}\right) \mathit{u2}_{4} \mathit{u2}_{1} + \frac{12}{5} \mathit{u2}_{4} \mathit{u3}_{2} + \left(-\frac{12

...

P_9 = \frac{36}{625} \mathit{u2}_{0} \mathit{u3}_{0}^{3} z_{0} + \left(-\frac{36}{625}\right) \mathit{u3}_{1} \mathit{u3}_{0}^{3} z_{0} + \frac{9}{625} \mathit{u3}_{0}^{4} z_{1} + \frac{18}{125} \mathit{u0}_{0} \mathit{u3}_{0}^{2} z_{0} + \frac{36}{125} \mathit{u1}_{0} \mathit{u2}_{0} \mathit{u3}_{0} z_{0} + \left(-\frac{36}{125}\right) \mathit{u1}_{0} \mathit{u3}_{1} \mathit{u3}_{0} z_{0} + \frac{18}{125} \mathit{u1}_{0} \mathit{u3}_{0}^{2} z_{1} + \left(-\frac{18}{125}\right) \mathit{u2}_{2} \mathit{u3}_{0}^{2} z_{0} + \left(-\frac{18}{125}\right) \mathit{u2}_{1} \mathit{u2}_{0} \mathit{u3}_{0} z_{0} + \left(-\frac{36}{125}\right) \mathit{u2}_{1} \mathit{u3}_{1} \mathit{u3}_{0} z_{0} + \frac{6}{125} \mathit{u2}_{0}^{3} z_{0} + \left(-\frac{18}{125}\right) \mathit{u2}_{0}^{2} \mathit{u3}_{1} z_{0} + \frac{18}{125} \mathit{u2}_{0}^{2} \mathit{u3}_{0} z_{1} + \left(-\frac{36}{125}\right) \mathit{u2}_{0} \mathit{u3}_{2} \mathit{u3}_{0} z_{0} + \left(-\frac{3}{25}\right) \mathit{u2}_{0} \mathit{u3}_{1}^{2} z_{0} + \left(-\frac{18}{125}\right) \mathit{u2}_{0} \mathit{u3}_{1} \mathit{u3}_{0} z_{1} + \frac{18}{125} \mathit{u2}_{0} \mathit{u3}_{0}^{2} z_{2} + \frac{18}{125} \mathit{u3}_{3} \mathit{u3}_{0}^{2} z_{0} + \frac{18}{25} \mathit{u3}_{2} \mathit{u3}_{1} \mathit{u3}_{0} z_{0} + \left(-\frac{18}{125}\right) \mathit{u3}_{2} \mathit{u3}_{0}^{2} z_{1} + \frac{27}{125} \mathit{u3}_{1}^{3} z_{0} + \left(-\frac{27}{125}\right) \mathit{u3}_{1}^{2} \mathit{u3}_{0} z_{1} + \frac{6}{125} \mathit{u3}_{0}^{3} z_{3} + \frac{18}{25} \mathit{u0}_{2} \mathit{u3}_{0} z_{0} + \frac{18}{25} \mathit{u0}_{1} \mathit{u2}_{0} z_{0} + \left(-\frac{12}{25}\right) \mathit{u0}_{1} \mathit{u3}_{1} z_{0} + \frac{36}{25} \mathit{u0}_{1} \mathit{u3}_{0} z_{1} + \frac{36}{25} \mathit{u0}_{0} \mathit{u1}_{0} z_{0} + \left(-\frac{18}{25}\right) \mathit{u0}_{0} \mathit{u2}_{1} z_{0} + \frac{36}{25} \mathit{u0}_{0} \mathit{u2}_{0} z_{1} + \left(-\frac{18}{25}\right) \mathit{u0}_{0} \mathit{u3}_{1} z_{1} + \frac{36}{25} \mathit{u0}_{0} \mathit{u3}_{0} z_{2} + \left(-\frac{6}{25}\right) \mathit{u1}_{2} \mathit{u2}_{0} z_{0} + \frac{6}{25} \mathit{u1}_{2} \mathit{u3}_{1} z_{0} + \frac{18}{25} \mathit{u1}_{2} \mathit{u3}_{0} z_{1} + \left(-\frac{12}{25}\right) \mathit{u1}_{1} \mathit{u2}_{1} z_{0} + \frac{18}{25} \mathit{u1}_{1} \mathit{u2}_{0} z_{1} + \frac{12}{25} \mathit{u1}_{1} \mathit{u3}_{2} z_{0} + \left(-\frac{18}{25}\right) \mathit{u1}_{1} \mathit{u3}_{1} z_{1} + \frac{36}{25} \mathit{u1}_{1} \mathit{u3}_{0} z_{2} + \frac{18}{25} \mathit{u1}_{0}^{2} z_{1} + \left(-\frac{12}{25}\right) \mathit{u1}_{0} \mathit{u2}_{2} z_{0} + \frac{36}{25} \mathit{u1}_{0} \mathit{u2}_{0} z_{2} + \frac{12}{25} \mathit{u1}_{0} \mathit{u3}_{3} z_{0} + \left(-\frac{12}{25}\right) \mathit{u1}_{0} \mathit{u3}_{2} z_{1} + \frac{36}{25} \mathit{u1}_{0} \mathit{u3}_{0} z_{3} + \frac{21}{125} \mathit{u2}_{4} \mathit{u3}_{0} z_{0} + \frac{3}{25} \mathit{u2}_{3} \mathit{u2}_{0} z_{0} + \frac{51}{125} \mathit{u2}_{3} \mathit{u3}_{1} z_{0} + \frac{12}{25} \mathit{u2}_{2} \mathit{u2}_{1} z_{0} + \left(-\frac{6}{25}\right) \mathit{u2}_{2} \mathit{u2}_{0} z_{1} + \frac{54}{125} \mathit{u2}_{2} \mathit{u3}_{2} z_{0} + \frac{3}{25} \mathit{u2}_{2} \mathit{u3}_{1} z_{1} + \frac{18}{25} \mathit{u2}_{2} \mathit{u3}_{0} z_{2} + \left(-\frac{9}{25}\right) \mathit{u2}_{1}^{2} z_{1} + \frac{18}{25} \mathit{u2}_{1} \mathit{u2}_{0} z_{2} + \frac{36}{125} \mathit{u2}_{1} \mathit{u3}_{3} z_{0} + \frac{9}{25} \mathit{u2}_{1} \mathit{u3}_{2} z_{1} + \left(-\frac{24}{25}\right) \mathit{u2}_{1} \mathit{u3}_{1} z_{2} + \frac{36}{25} \mathit{u2}_{1} \mathit{u3}_{0} z_{3} + \frac{18}{25} \mathit{u2}_{0}^{2} z_{3} + \frac{27}{125} \mathit{u2}_{0} \mathit{u3}_{4} z_{0} + \frac{3}{25} \mathit{u2}_{0} \mathit{u3}_{3} z_{1} + \left(-\frac{6}{25}\right) \mathit{u2}_{0} \mathit{u3}_{2} z_{2} + \frac{18}{25} \mathit{u2}_{0} \mathit{u3}_{1} z_{3} + \frac{36}{25} \mathit{u2}_{0} \mathit{u3}_{0} z_{4} + \left(-\frac{21}{12

\left\{\begin{array}{rcl}f_{1,0} & = & \left(-\frac{9}{625}\right) \mathit{u0}_{1} \mathit{u3}_{0}^{4} + \frac{36}{625} \mathit{u1}_{0} \mathit{u2}_{1} \mathit{u3}_{0}^{3} + \frac{108}{625} \mathit{u1}_{0} \mathit{u2}_{0} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \left(-\frac{36}{625}\right) \mathit{u1}_{0} \mathit{u3}_{2} \mathit{u3}_{0}^{3} + \left(-\frac{108}{625}\right) \mathit{u1}_{0} \mathit{u3}_{1}^{2} \mathit{u3}_{0}^{2} + \frac{36}{625} \mathit{u2}_{3} \mathit{u3}_{0}^{4} + \frac{36}{625} \mathit{u2}_{2} \mathit{u2}_{0} \mathit{u3}_{0}^{3} + \frac{324}{625} \mathit{u2}_{2} \mathit{u3}_{1} \mathit{u3}_{0}^{3} + \frac{216}{625} \mathit{u2}_{1} \mathit{u2}_{0} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \frac{324}{625} \mathit{u2}_{1} \mathit{u3}_{2} \mathit{u3}_{0}^{3} + \frac{648}{625} \mathit{u2}_{1} \mathit{u3}_{1}^{2} \mathit{u3}_{0}^{2} + \frac{108}{625} \mathit{u2}_{0}^{2} \mathit{u3}_{2} \mathit{u3}_{0}^{2} + \frac{216}{625} \mathit{u2}_{0}^{2} \mathit{u3}_{1}^{2} \mathit{u3}_{0} + \frac{72}{625} \mathit{u2}_{0} \mathit{u3}_{3} \mathit{u3}_{0}^{3} + \frac{324}{625} \mathit{u2}_{0} \mathit{u3}_{2} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \left(-\frac{36}{625}\right) \mathit{u3}_{4} \mathit{u3}_{0}^{4} + \left(-\frac{432}{625}\right) \mathit{u3}_{3} \mathit{u3}_{1} \mathit{u3}_{0}^{3} + \left(-\frac{324}{625}\right) \mathit{u3}_{2}^{2} \mathit{u3}_{0}^{3} + \left(-\frac{1296}{625}\right) \mathit{u3}_{2} \mathit{u3}_{1}^{2} \mathit{u3}_{0}^{2} + \left(-\frac{216}{625}\right) \mathit{u3}_{1}^{4} \mathit{u3}_{0} + \frac{12}{125} \mathit{u0}_{3} \mathit{u3}_{0}^{3} + \frac{108}{125} \mathit{u0}_{2} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \left(-\frac{18}{125}\right) \mathit{u0}_{1} \mathit{u2}_{0}^{2} \mathit{u3}_{0} + \frac{18}{25} \mathit{u0}_{1} \mathit{u2}_{0} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{126}{125} \mathit{u0}_{1} \mathit{u3}_{2} \mathit{u3}_{0}^{2} + \frac{27}{25} \mathit{u0}_{1} \mathit{u3}_{1}^{2} \mathit{u3}_{0} + \frac{36}{125} \mathit{u0}_{0} \mathit{u1}_{0} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{36}{125} \mathit{u0}_{0} \mathit{u2}_{0} \mathit{u3}_{2} \mathit{u3}_{0} + \frac{36}{125} \mathit{u0}_{0} \mathit{u2}_{0} \mathit{u3}_{1}^{2} + \frac{36}{125} \mathit{u0}_{0} \mathit{u3}_{3} \mathit{u3}_{0}^{2} + \frac{108}{125} \mathit{u0}_{0} \mathit{u3}_{2} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{36}{125} \mathit{u1}_{3} \mathit{u2}_{0} \mathit{u3}_{0}^{2} + \left(-\frac{36}{125}\right) \mathit{u1}_{3} \mathit{u3}_{1} \mathit{u3}_{0}^{2} + \frac{108}{125} \mathit{u1}_{2} \mathit{u2}_{1} \mathit{u3}_{0}^{2} + \frac{36}{125} \mathit{u1}_{2} \mathit{u2}_{0}^{2} \mathit{u3}_{0} + \frac{72}{125} \mathit{u1}_{2} \mathit{u2}_{0} \mathit{u3}_{1} \mathit{u3}_{0} + \left(-\frac{108}{125}\right) \mathit{u1}_{2} \mathit{u3}_{2} \mathit{u3}_{0}^{2} + \left(-\frac{108}{125}\right) \mathit{u1}_{2} \mathit{u3}_{1}^{2} \mathit{u3}_{0} + \frac{36}{125} \mathit{u1}_{1} \mathit{u1}_{0} \mathit{u2}_{0} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u1}_{1} \mathit{u1}_{0} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{108}{125} \mathit{u1}_{1} \mathit{u2}_{2} \mathit{u3}_{0}^{2} + \frac{72}{125} \mathit{u1}_{1} \mathit{u2}_{1} \mathit{u2}_{0} \mathit{u3}_{0} + \frac{216}{125} \mathit{u1}_{1} \mathit{u2}_{1} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{72}{125} \mathit{u1}_{1} \mathit{u2}_{0}^{2} \mathit{u3}_{1} + \frac{36}{125} \mathit{u1}_{1} \mathit{u2}_{0} \mathit{u3}_{2} \mathit{u3}_{0} + \left(-\frac{72}{125}\right) \mathit{u1}_{1} \mathit{u2}_{0} \mathit{u3}_{1}^{2} + \left(-\frac{108}{125}\right) \mathit{u1}_{1} \mathit{u3}_{3} \mathit{u3}_{0}^{2} + \left(-\frac{324}{125}\right) \mathit{u1}_{1} \mathit{u3}_{2} \mathit{u3}_{1} \mathit{u3}_{0} + \frac{36}{125} \mathit{u1}_{0}^{2} \mathit{u2}_{1} \mathit{u3}_{0} + \frac{36}{125} \mathit{u1}_{0}^{2} \mathit{u2}_{0} \mathit{u3}_{1} + \left(-\frac{36}{125}\right) \mathit{u1}_{0}^{2} \mathit{u3}_{2} \mathit{u3}_{0} + \left(-\frac{36}{125}\right) \mathit{u1}_{0}^{2} \mathit{u3}_{1}^{2} + \fra

-------------------------------------------------------------------------------------------------------------------
Computed the almost commutator of order 10


P_10 = 1 \mathit{u0}_{3} \mathit{u3}_{0} z_{0} + 1 \mathit{u0}_{2} \mathit{u2}_{0} z_{0} + 3 \mathit{u0}_{2} \mathit{u3}_{0} z_{1} + 1 \mathit{u0}_{1} \mathit{u1}_{0} z_{0} + 2 \mathit{u0}_{1} \mathit{u2}_{0} z_{1} + 3 \mathit{u0}_{1} \mathit{u3}_{0} z_{2} + 1 \mathit{u0}_{0}^{2} z_{0} + 2 \mathit{u0}_{0} \mathit{u1}_{0} z_{1} + 2 \mathit{u0}_{0} \mathit{u2}_{0} z_{2} + 2 \mathit{u0}_{0} \mathit{u3}_{0} z_{3} + 1 \mathit{u1}_{3} \mathit{u3}_{0} z_{1} + 1 \mathit{u1}_{2} \mathit{u2}_{0} z_{1} + 3 \mathit{u1}_{2} \mathit{u3}_{0} z_{2} + 1 \mathit{u1}_{1} \mathit{u1}_{0} z_{1} + 2 \mathit{u1}_{1} \mathit{u2}_{0} z_{2} + 3 \mathit{u1}_{1} \mathit{u3}_{0} z_{3} + 1 \mathit{u1}_{0}^{2} z_{2} + 1 \mathit{u1}_{0} \mathit{u2}_{1} z_{2} + 2 \mathit{u1}_{0} \mathit{u2}_{0} z_{3} + 1 \mathit{u1}_{0} \mathit{u3}_{1} z_{3} + 2 \mathit{u1}_{0} \mathit{u3}_{0} z_{4} + 1 \mathit{u2}_{3} \mathit{u3}_{0} z_{2} + 1 \mathit{u2}_{2} \mathit{u2}_{0} z_{2} + 3 \mathit{u2}_{2} \mathit{u3}_{0} z_{3} + 2 \mathit{u2}_{1} \mathit{u2}_{0} z_{3} + 3 \mathit{u2}_{1} \mathit{u3}_{0} z_{4} + 1 \mathit{u2}_{0}^{2} z_{4} + 1 \mathit{u2}_{0} \mathit{u3}_{2} z_{3} + 2 \mathit{u2}_{0} \mathit{u3}_{1} z_{4} + 2 \mathit{u2}_{0} \mathit{u3}_{0} z_{5} + 1 \mathit{u3}_{3} \mathit{u3}_{0} z_{3} + 3 \mathit{u3}_{2} \mathit{u3}_{0} z_{4} + 3 \mathit{u3}_{1} \mathit{u3}_{0} z_{5} + 1 \mathit{u3}_{0}^{2} z_{6} - \mathit{u0}_{5} z_{0} + \left(-5\right) \mathit{u0}_{4} z_{1} + \left(-10\right) \mathit{u0}_{3} z_{2} + \left(-10\right) \mathit{u0}_{2} z_{3} + \left(-5\right) \mathit{u0}_{1} z_{4} + \left(-2\right) \mathit{u0}_{0} z_{5} - \mathit{u1}_{5} z_{1} + \left(-5\right) \mathit{u1}_{4} z_{2} + \left(-10\right) \mathit{u1}_{3} z_{3} + \left(-10\right) \mathit{u1}_{2} z_{4} + \left(-5\right) \mathit{u1}_{1} z_{5} + \left(-2\right) \mathit{u1}_{0} z_{6} - \mathit{u2}_{5} z_{2} + \left(-5\right) \mathit{u2}_{4} z_{3} + \left(-10\right) \mathit{u2}_{3} z_{4} + \left(-10\right) \mathit{u2}_{2} z_{5} + \left(-5\right) \mathit{u2}_{1} z_{6} + \left(-2\right) \mathit{u2}_{0} z_{7} - \mathit{u3}_{5} z_{3} + \left(-5\right) \mathit{u3}_{4} z_{4} + \left(-10\right) \mathit{u3}_{3} z_{5} + \left(-10\right) \mathit{u3}_{2} z_{6} + \left(-5\right) \mathit{u3}_{1} z_{7} + \left(-2\right) \mathit{u3}_{0} z_{8} + 1 z_{10}

... This commutes with L
CPU times: user 25.4 s, sys: 160 ms, total: 25.6 s
Wall time: 25.6 s
